# Știința Datelor în Cloud: Metoda "Azure ML SDK"

## Introducere

În acest notebook, vom învăța cum să utilizăm Azure ML SDK pentru a antrena, implementa și utiliza un model prin Azure ML.

Pre-rechizite:
1. Ai creat un workspace Azure ML.
2. Ai încărcat [setul de date Heart Failure](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) în Azure ML.
3. Ai încărcat acest notebook în Azure ML Studio.

Pașii următori sunt:

1. Crearea unui Experiment într-un Workspace existent.
2. Crearea unui cluster de Compute.
3. Încărcarea setului de date.
4. Configurarea AutoML folosind AutoMLConfig.
5. Rularea experimentului AutoML.
6. Explorarea rezultatelor și obținerea celui mai bun model.
7. Înregistrarea celui mai bun model.
8. Implementarea celui mai bun model.
9. Utilizarea endpoint-ului.

## Importuri specifice Azure Machine Learning SDK


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## Inițializare Spațiu de Lucru  
Inițializează un obiect spațiu de lucru din configurația salvată. Asigură-te că fișierul de configurare este prezent la .\config.json  


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Crearea unui experiment Azure ML

Să creăm un experiment numit 'aml-experiment' în workspace-ul pe care tocmai l-am inițializat.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## Crearea unui Cluster de Calcul
Va trebui să creați o [țintă de calcul](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) pentru rularea AutoML.


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## Date
Asigură-te că ai încărcat setul de date în Azure ML și că cheia are același nume ca setul de date.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## Configurare AutoML


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## Execuție AutoML


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## Implementarea celui mai bun model

Rulați următorul cod pentru a implementa cel mai bun model. Puteți vedea starea implementării în portalul Azure ML. Acest pas poate dura câteva minute.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## Consumă Endpoint-ul
Poți adăuga intrări la exemplul de intrare de mai jos.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**Declinarea responsabilității**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși depunem eforturi pentru a asigura acuratețea, vă rugăm să rețineți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa nativă ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea realizată de un profesionist uman. Nu ne asumăm răspunderea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
